In [ ]:
import os
import sys

if "google.colab" in sys.modules:
    !git clone https://github.com/Sneha73685/f1-tyre-strategy-colab.git
    %cd f1-tyre-strategy-colab
    !pip install -r requirements.txt

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
!pwd
!ls

In [ ]:
import os

os.makedirs("outputs", exist_ok=True)

In [ ]:
TRACK_DEGRADATION_FACTORS = {
    # Low degradation / street or smooth surface
    "Monaco": 0.7,
    "Baku": 0.8,
    "Monza": 0.8,

    # Medium degradation
    "Silverstone": 1.0,
    "Mexico": 1.0,
    "AbuDhabi": 1.0,

    # Medium-high degradation
    "Barcelona": 1.1,
    "Singapore": 1.1,

    # High degradation
    "Spa": 1.2,
    "Suzuka": 1.2,

    # Very high degradation
    "Bahrain": 1.3,
    "Hungaroring": 1.3
}

In [ ]:
DRIVER_STYLE_FACTORS = {
    "Aggressive": 1.1,
    "Balanced": 1.0,
    "TyreSaver": 0.9
}

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
import sys
sys.path.append("src")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from data_loader import load_race_data
from degradation import TyreDegradationModel
from strategy import one_stop_strategy, two_stop_strategy

In [ ]:
def create_track_aware_model(compound, track):
    model = TyreDegradationModel(compound=compound)
    track_factor = TRACK_DEGRADATION_FACTORS.get(track, 1.0)
    model.alpha *= track_factor
    return model

In [ ]:
race_data = load_race_data("data/processed/race_data.csv")
race_data.head()

In [ ]:
import inspect
from degradation import TyreDegradationModel
print(inspect.signature(TyreDegradationModel.predict))

In [ ]:
model = TyreDegradationModel(compound="Medium")
model.fit(race_data)

In [ ]:
tyre_life = np.arange(1, 31)
curve = model.predict(tyre_life)

plt.figure()
plt.plot(tyre_life, curve)
plt.xlabel("Tyre Life (laps)")
plt.ylabel("Relative Performance")
plt.title("Tyre Degradation vs Tyre Life")
plt.show()

In [ ]:
tyre_life = np.arange(1, 31)

compounds = ["Soft", "Medium", "Hard"]

plt.figure(figsize=(8,5))

for compound in compounds:
    model = TyreDegradationModel(compound=compound)
    curve = model.predict(tyre_life)
    plt.plot(tyre_life, curve, label=compound)

plt.xlabel("Tyre Life (laps)")
plt.ylabel("Relative Performance")
plt.title("Tyre Degradation by Compound")
plt.legend()
plt.show()

In [ ]:
from strategy import one_stop_strategy

total_laps = 70
model = TyreDegradationModel(compound="Medium")

pit_range = range(15, 41)
one_stop_results = []

for pit in pit_range:
    cost = one_stop_strategy(model, total_laps, pit)
    one_stop_results.append((pit, cost))

one_stop_df = pd.DataFrame(one_stop_results, columns=["Pit Lap", "Total Degradation Cost"])
one_stop_df.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(one_stop_df["Pit Lap"], one_stop_df["Total Degradation Cost"])
plt.xlabel("Pit Stop Lap")
plt.ylabel("Total Degradation Cost")
plt.title("One-Stop Strategy Cost (Medium Compound)")
plt.show()

In [ ]:
from strategy import two_stop_strategy

two_stop_results = []

for pit1 in range(15, 30):
    for pit2 in range(pit1 + 10, 50):
        cost = two_stop_strategy(model, total_laps, pit1, pit2)
        two_stop_results.append((pit1, pit2, cost))

two_stop_df = pd.DataFrame(
    two_stop_results,
    columns=["Pit 1", "Pit 2", "Total Degradation Cost"]
)

two_stop_df.sort_values("Total Degradation Cost").head()

In [ ]:
best_one_stop = one_stop_df.loc[
    one_stop_df["Total Degradation Cost"].idxmin()
]

best_one_stop

In [ ]:
best_two_stop = two_stop_df.loc[
    two_stop_df["Total Degradation Cost"].idxmin()
]

best_two_stop

In [ ]:
comparison_df = pd.DataFrame([
    ["One-stop", best_one_stop["Total Degradation Cost"]],
    ["Two-stop", best_two_stop["Total Degradation Cost"]],
], columns=["Strategy", "Total Degradation Cost"])

comparison_df

In [ ]:
compounds = ["Soft", "Medium", "Hard"]
total_laps = 70
pit_range = range(15, 41)

compound_one_stop_results = []

for compound in compounds:
    model = TyreDegradationModel(compound=compound)
    
    for pit in pit_range:
        cost = one_stop_strategy(model, total_laps, pit)
        compound_one_stop_results.append(
            (compound, pit, cost)
        )

compound_one_stop_df = pd.DataFrame(
    compound_one_stop_results,
    columns=["Compound", "Pit Lap", "Total Degradation Cost"]
)

compound_one_stop_df.head()

In [ ]:
best_one_stop_by_compound = (
    compound_one_stop_df
    .loc[compound_one_stop_df.groupby("Compound")["Total Degradation Cost"].idxmin()]
)

best_one_stop_by_compound

In [ ]:
plt.figure(figsize=(8,5))

for compound in compounds:
    subset = compound_one_stop_df[
        compound_one_stop_df["Compound"] == compound
    ]
    plt.plot(
        subset["Pit Lap"],
        subset["Total Degradation Cost"],
        label=compound
    )

plt.xlabel("Pit Stop Lap")
plt.ylabel("Total Degradation Cost")
plt.title("One-Stop Strategy Cost by Compound")
plt.legend()
plt.show()

In [ ]:
compound_two_stop_results = []

for compound in compounds:
    model = TyreDegradationModel(compound=compound)
    
    for pit1 in range(15, 30):
        for pit2 in range(pit1 + 10, 50):
            cost = two_stop_strategy(model, total_laps, pit1, pit2)
            compound_two_stop_results.append(
                (compound, pit1, pit2, cost)
            )

compound_two_stop_df = pd.DataFrame(
    compound_two_stop_results,
    columns=["Compound", "Pit 1", "Pit 2", "Total Degradation Cost"]
)

compound_two_stop_df.head()

In [ ]:
best_two_stop_by_compound = (
    compound_two_stop_df
    .loc[compound_two_stop_df.groupby("Compound")["Total Degradation Cost"].idxmin()]
)

best_two_stop_by_compound

In [ ]:
import numpy as np

def monte_carlo_strategy(
    strategy_fn,
    compound,
    total_laps,
    pit_params,
    alpha_mean,
    alpha_std,
    n_sim=1000
):
    costs = []

    for _ in range(n_sim):
        sampled_alpha = np.random.normal(alpha_mean, alpha_std)

        model = TyreDegradationModel(compound=compound)

        model.alpha = sampled_alpha

        cost = strategy_fn(model, total_laps, *pit_params)
        costs.append(cost)

    costs = np.array(costs)

    return {
        "mean_cost": costs.mean(),
        "std_cost": costs.std(),
        "ci_lower": np.percentile(costs, 2.5),
        "ci_upper": np.percentile(costs, 97.5)
    }

In [ ]:
mc_one_stop = monte_carlo_strategy(
    strategy_fn=one_stop_strategy,
    compound="Soft",
    total_laps=70,
    pit_params=(35,),
    alpha_mean=0.03,
    alpha_std=0.005,
    n_sim=2000
)

mc_two_stop = monte_carlo_strategy(
    strategy_fn=two_stop_strategy,
    compound="Soft",
    total_laps=70,
    pit_params=(23, 46),
    alpha_mean=0.03,
    alpha_std=0.005,
    n_sim=2000
)

mc_one_stop, mc_two_stop

In [ ]:
mc_one_stop_medium = monte_carlo_strategy(
    strategy_fn=one_stop_strategy,
    compound="Medium",
    total_laps=70,
    pit_params=(best_one_stop_by_compound.loc[
        best_one_stop_by_compound["Compound"] == "Medium", "Pit Lap"
    ].values[0],),
    alpha_mean=0.03,
    alpha_std=0.005,
    n_sim=2000
)

mc_two_stop_medium = monte_carlo_strategy(
    strategy_fn=two_stop_strategy,
    compound="Medium",
    total_laps=70,
    pit_params=(
        best_two_stop_by_compound.loc[
            best_two_stop_by_compound["Compound"] == "Medium", "Pit 1"
        ].values[0],
        best_two_stop_by_compound.loc[
            best_two_stop_by_compound["Compound"] == "Medium", "Pit 2"
        ].values[0],
    ),
    alpha_mean=0.03,
    alpha_std=0.005,
    n_sim=2000
)

mc_one_stop_medium, mc_two_stop_medium

In [ ]:
mc_one_stop_hard = monte_carlo_strategy(
    strategy_fn=one_stop_strategy,
    compound="Hard",
    total_laps=70,
    pit_params=(best_one_stop_by_compound.loc[
        best_one_stop_by_compound["Compound"] == "Hard", "Pit Lap"
    ].values[0],),
    alpha_mean=0.02,
    alpha_std=0.004,
    n_sim=2000
)

mc_two_stop_hard = monte_carlo_strategy(
    strategy_fn=two_stop_strategy,
    compound="Hard",
    total_laps=70,
    pit_params=(
        best_two_stop_by_compound.loc[
            best_two_stop_by_compound["Compound"] == "Hard", "Pit 1"
        ].values[0],
        best_two_stop_by_compound.loc[
            best_two_stop_by_compound["Compound"] == "Hard", "Pit 2"
        ].values[0],
    ),
    alpha_mean=0.02,
    alpha_std=0.004,
    n_sim=2000
)

mc_one_stop_hard, mc_two_stop_hard

In [ ]:
mc_summary = pd.DataFrame([
    ["Soft", "One-stop", mc_one_stop["mean_cost"], mc_one_stop["std_cost"],
     mc_one_stop["ci_lower"], mc_one_stop["ci_upper"]],
    ["Soft", "Two-stop", mc_two_stop["mean_cost"], mc_two_stop["std_cost"],
     mc_two_stop["ci_lower"], mc_two_stop["ci_upper"]],

    ["Medium", "One-stop", mc_one_stop_medium["mean_cost"], mc_one_stop_medium["std_cost"],
     mc_one_stop_medium["ci_lower"], mc_one_stop_medium["ci_upper"]],
    ["Medium", "Two-stop", mc_two_stop_medium["mean_cost"], mc_two_stop_medium["std_cost"],
     mc_two_stop_medium["ci_lower"], mc_two_stop_medium["ci_upper"]],

    ["Hard", "One-stop", mc_one_stop_hard["mean_cost"], mc_one_stop_hard["std_cost"],
     mc_one_stop_hard["ci_lower"], mc_one_stop_hard["ci_upper"]],
    ["Hard", "Two-stop", mc_two_stop_hard["mean_cost"], mc_two_stop_hard["std_cost"],
     mc_two_stop_hard["ci_lower"], mc_two_stop_hard["ci_upper"]],
],
columns=["Compound", "Strategy", "Mean Cost", "Std Dev", "CI Lower", "CI Upper"])

mc_summary

In [ ]:
plt.figure(figsize=(8,5))

for compound in ["Soft", "Medium", "Hard"]:
    subset = mc_summary[mc_summary["Compound"] == compound]
    plt.bar(
        [f"{compound}-{s}" for s in subset["Strategy"]],
        subset["Mean Cost"],
        label=compound
    )

plt.ylabel("Mean Degradation Cost")
plt.title("Monte Carlo Strategy Comparison Across Compounds")
plt.show()

In [ ]:
mc_summary.to_csv("monte_carlo_strategy_summary.csv", index=False)
mc_summary

In [ ]:
plt.figure(figsize=(9,5))

for compound in ["Soft", "Medium", "Hard"]:
    subset = mc_summary[mc_summary["Compound"] == compound]
    plt.plot(
        subset["Strategy"],
        subset["Mean Cost"],
        marker="o",
        label=compound
    )

plt.ylabel("Mean Degradation Cost")
plt.title("Monte Carlo Strategy Comparison Across Compounds")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("Monte Carlo analysis completed successfully.")
print("Results saved to monte_carlo_strategy_summary.csv")

Layer 2 code begins from here
Layer 2: Context-Aware Strategy Insights

Extending the baseline framework, track- and driver-aware parameterization reveals that optimal pit strategies are sensitive to contextual factors.
High-degradation tracks amplify the benefit of multi-stop strategies, while driver behaviour further modulates degradation outcomes.
These results demonstrate that the proposed framework generalizes beyond a fixed race scenario and can support context-dependent strategy evaluation.


In [ ]:
def monte_carlo_strategy_track_aware(
    strategy_fn,
    compound,
    track,
    total_laps,
    pit_params,
    alpha_std,
    n_sim=1000
):
    base_model = TyreDegradationModel(compound=compound)
    base_alpha = base_model.alpha * TRACK_DEGRADATION_FACTORS.get(track, 1.0)

    return monte_carlo_strategy(
        strategy_fn=strategy_fn,
        compound=compound,
        total_laps=total_laps,
        pit_params=pit_params,
        alpha_mean=base_alpha,
        alpha_std=alpha_std,
        n_sim=n_sim
    )

In [ ]:
track = "Monaco"

mc_soft_one_stop_monaco = monte_carlo_strategy_track_aware(
    strategy_fn=one_stop_strategy,
    compound="Soft",
    track=track,
    total_laps=70,
    pit_params=(35,),
    alpha_std=0.005,
    n_sim=2000
)

mc_soft_two_stop_monaco = monte_carlo_strategy_track_aware(
    strategy_fn=two_stop_strategy,
    compound="Soft",
    track=track,
    total_laps=70,
    pit_params=(23, 46),
    alpha_std=0.005,
    n_sim=2000
)

mc_soft_one_stop_monaco, mc_soft_two_stop_monaco

In [ ]:
track = "Hungary"

mc_soft_one_stop_hungary = monte_carlo_strategy_track_aware(
    strategy_fn=one_stop_strategy,
    compound="Soft",
    track=track,
    total_laps=70,
    pit_params=(35,),
    alpha_std=0.005,
    n_sim=2000
)

mc_soft_two_stop_hungary = monte_carlo_strategy_track_aware(
    strategy_fn=two_stop_strategy,
    compound="Soft",
    track=track,
    total_laps=70,
    pit_params=(23, 46),
    alpha_std=0.005,
    n_sim=2000
)

mc_soft_one_stop_hungary, mc_soft_two_stop_hungary

In [ ]:
track_summary = pd.DataFrame([
    ["Monaco", "One-stop", mc_soft_one_stop_monaco["mean_cost"]],
    ["Monaco", "Two-stop", mc_soft_two_stop_monaco["mean_cost"]],
    ["Hungary", "One-stop", mc_soft_one_stop_hungary["mean_cost"]],
    ["Hungary", "Two-stop", mc_soft_two_stop_hungary["mean_cost"]],
], columns=["Track", "Strategy", "Mean Cost"])

track_summary

In [ ]:
def create_context_aware_model(compound, track, driver_style):
    model = TyreDegradationModel(compound=compound)

    model.alpha *= TRACK_DEGRADATION_FACTORS.get(track, 1.0)
    model.alpha *= DRIVER_STYLE_FACTORS.get(driver_style, 1.0)

    return model

In [ ]:
def monte_carlo_strategy_context_aware(
    strategy_fn,
    compound,
    track,
    driver_style,
    total_laps,
    pit_params,
    alpha_std,
    n_sim=1000
):
    base_model = create_context_aware_model(compound, track, driver_style)
    base_alpha = base_model.alpha

    return monte_carlo_strategy(
        strategy_fn=strategy_fn,
        compound=compound,
        total_laps=total_laps,
        pit_params=pit_params,
        alpha_mean=base_alpha,
        alpha_std=alpha_std,
        n_sim=n_sim
    )

In [ ]:
track = "Hungary"

mc_aggressive = monte_carlo_strategy_context_aware(
    strategy_fn=two_stop_strategy,
    compound="Soft",
    track=track,
    driver_style="Aggressive",
    total_laps=70,
    pit_params=(23, 46),
    alpha_std=0.005,
    n_sim=2000
)

mc_tyre_saver = monte_carlo_strategy_context_aware(
    strategy_fn=two_stop_strategy,
    compound="Soft",
    track=track,
    driver_style="TyreSaver",
    total_laps=70,
    pit_params=(23, 46),
    alpha_std=0.005,
    n_sim=2000
)

mc_aggressive, mc_tyre_saver

In [ ]:
driver_summary = pd.DataFrame([
    ["Aggressive", mc_aggressive["mean_cost"]],
    ["TyreSaver", mc_tyre_saver["mean_cost"]],
], columns=["Driver Style", "Mean Cost"])

driver_summary

Layer 3 Code Begins Here

In [ ]:
def strategy_win_probability(costs_a, costs_b):
    return np.mean(costs_a < costs_b)

In [ ]:
def monte_carlo_strategy_samples(
    strategy_fn,
    compound,
    alpha_mean,
    alpha_std,
    total_laps,
    pit_params,
    n_sim=1000
):
    costs = []

    for _ in range(n_sim):
        model = TyreDegradationModel(compound=compound)
        model.alpha = np.random.normal(alpha_mean, alpha_std)
        cost = strategy_fn(model, total_laps, *pit_params)
        costs.append(cost)

    return np.array(costs)

In [ ]:
alpha_base = create_context_aware_model(
    compound="Soft",
    track="Hungary",
    driver_style="Balanced"
).alpha

one_stop_costs = monte_carlo_strategy_samples(
    strategy_fn=one_stop_strategy,
    compound="Soft",
    alpha_mean=alpha_base,
    alpha_std=0.005,
    total_laps=70,
    pit_params=(35,),
    n_sim=3000
)

two_stop_costs = monte_carlo_strategy_samples(
    strategy_fn=two_stop_strategy,
    compound="Soft",
    alpha_mean=alpha_base,
    alpha_std=0.005,
    total_laps=70,
    pit_params=(23, 46),
    n_sim=3000
)

In [ ]:
win_prob = strategy_win_probability(two_stop_costs, one_stop_costs)

expected_regret = np.mean(
    np.maximum(two_stop_costs - one_stop_costs, 0)
)

risk_averse_cost = np.mean(two_stop_costs) + 0.5 * np.std(two_stop_costs)

win_prob, expected_regret, risk_averse_cost

In [ ]:
decision_summary = pd.DataFrame([
    ["One-stop", one_stop_costs.mean(), one_stop_costs.std()],
    ["Two-stop", two_stop_costs.mean(), two_stop_costs.std()],
], columns=["Strategy", "Mean Cost", "Std Dev"])

decision_summary

Layer 3: Risk-Aware Strategy Selection

While two-stop strategies demonstrate lower expected degradation cost, Monte Carlo sampling enables evaluation of risk and robustness.
Decision metrics such as win probability and expected regret provide additional insight beyond mean performance, supporting risk-aware strategy selection under uncertainty.

In [ ]:
TRACKS = [
    "Monaco", "Singapore", "Baku", "Bahrain", "Barcelona",
    "Silverstone", "Monza", "Spa", "Mexico", "AbuDhabi",
    "Suzuka", "Hungaroring"
]

DRIVERS = [
    "Max Verstappen", "Lewis Hamilton", "Charles Leclerc",
    "Carlos Sainz", "Lando Norris", "Sergio Pérez",
    "George Russell", "Fernando Alonso", "Esteban Ocon",
    "Pierre Gasly"
]

In [ ]:
DRIVER_STYLE_MAP = {
    "Max Verstappen": "Aggressive",
    "Lewis Hamilton": "TyreSaver",
    "Charles Leclerc": "Aggressive",
    "Carlos Sainz": "Balanced",
    "Lando Norris": "Balanced",
    "Sergio Pérez": "TyreSaver",
    "George Russell": "Balanced",
    "Fernando Alonso": "TyreSaver",
    "Esteban Ocon": "Balanced",
    "Pierre Gasly": "Balanced"
}

In [ ]:
batch_results = []

for track in TRACKS:
    for driver in DRIVERS:
        style = DRIVER_STYLE_MAP[driver]

        alpha_base = create_context_aware_model(
            compound="Soft",
            track=track,
            driver_style=style
        ).alpha

        costs = monte_carlo_strategy_samples(
            strategy_fn=two_stop_strategy,
            compound="Soft",
            alpha_mean=alpha_base,
            alpha_std=0.005,
            total_laps=70,
            pit_params=(23, 46),
            n_sim=1500
        )

        batch_results.append([
            track,
            driver,
            style,
            costs.mean(),
            costs.std()
        ])

batch_df = pd.DataFrame(
    batch_results,
    columns=["Track", "Driver", "Driver Style", "Mean Cost", "Std Dev"]
)

batch_df.head()

In [ ]:
reference_cost = batch_df["Mean Cost"].median()

batch_df["Driver Degradation Index"] = (
    batch_df["Mean Cost"] / reference_cost
)

batch_df.head()

In [ ]:
driver_ddi = (
    batch_df
    .groupby("Driver")["Driver Degradation Index"]
    .mean()
    .reset_index()
    .sort_values("Driver Degradation Index")
)

driver_ddi

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(driver_ddi["Driver"], driver_ddi["Driver Degradation Index"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("Driver Degradation Index")
plt.title("Data-Driven Driver Degradation Tendencies (2020–2024)")
plt.axhline(1.0, linestyle="--")
plt.show()

In [ ]:
def validate_inputs(track, driver, compound):
    if track not in TRACK_DEGRADATION_FACTORS:
        raise ValueError(f"Unknown track: {track}")

    if driver not in DRIVER_STYLE_MAP:
        raise ValueError(f"Unknown driver: {driver}")

    if compound not in ["Soft", "Medium", "Hard"]:
        raise ValueError(f"Unsupported compound: {compound}")

In [ ]:
validate_inputs(track, driver, compound)
def predict_race_strategy(track, driver, compound="Soft",
                          total_laps=70,
                          one_stop_pit=35,
                          two_stop_pits=(23, 46),
                          alpha_std=0.005,
                          n_sim=3000):

    style = DRIVER_STYLE_MAP[driver]

    alpha_base = create_context_aware_model(
        compound=compound,
        track=track,
        driver_style=style
    ).alpha

    one_costs = monte_carlo_strategy_samples(
        strategy_fn=one_stop_strategy,
        compound=compound,
        alpha_mean=alpha_base,
        alpha_std=alpha_std,
        total_laps=total_laps,
        pit_params=(one_stop_pit,),
        n_sim=n_sim
    )

    two_costs = monte_carlo_strategy_samples(
        strategy_fn=two_stop_strategy,
        compound=compound,
        alpha_mean=alpha_base,
        alpha_std=alpha_std,
        total_laps=total_laps,
        pit_params=two_stop_pits,
        n_sim=n_sim
    )

    win_prob = np.mean(two_costs < one_costs)

    result = pd.DataFrame([
        ["One-stop", one_costs.mean(), one_costs.std()],
        ["Two-stop", two_costs.mean(), two_costs.std()],
    ], columns=["Strategy", "Mean Cost", "Std Dev"])

    best_strategy = "Two-stop" if two_costs.mean() < one_costs.mean() else "One-stop"

    return {
        "track": track,
        "driver": driver,
        "best_strategy": best_strategy,
        "two_stop_win_probability": win_prob,
        "comparison_table": result
    }

In [ ]:
example = predict_race_strategy(
    track="Hungaroring",
    driver="Lewis Hamilton"
)

example["best_strategy"], example["two_stop_win_probability"], example["comparison_table"]

In [ ]:
def run_full_grid():
    outputs = []

    for track in TRACKS:
        for driver in DRIVERS:
            res = predict_race_strategy(track, driver)

            outputs.append([
                track,
                driver,
                res["best_strategy"],
                res["two_stop_win_probability"]
            ])

    return pd.DataFrame(
        outputs,
        columns=["Track", "Driver", "Best Strategy", "Two-stop Win Prob"]
    )

In [ ]:
full_results = run_full_grid()
full_results.head()

In [ ]:
full_results.to_csv("outputs/final_strategy_predictions.csv", index=False)

In [ ]:
def run_example():
    res = predict_race_strategy("Hungaroring", "Lewis Hamilton")

    print("Best Strategy:", res["best_strategy"])
    print("Two-stop Win Probability:", round(res["two_stop_win_probability"], 3))
    print("\nComparison:\n")
    print(res["comparison_table"])

PHASE 2

In [ ]:
BASE_LAP_TIMES = {
    "Monaco": 74,
    "Singapore": 102,
    "Baku": 105,
    "Bahrain": 95,
    "Barcelona": 96,
    "Silverstone": 88,
    "Monza": 80,
    "Spa": 105,
    "Mexico": 78,
    "AbuDhabi": 92,
    "Suzuka": 90,
    "Hungaroring": 98
}

In [ ]:
def fuel_time_penalty(lap, total_laps, max_penalty=2.5):
    fuel_fraction = 1 - (lap - 1) / total_laps
    return max_penalty * fuel_fraction

In [ ]:
def tyre_warmup_penalty(lap_in_stint, warmup_laps=2, penalty=1.2):
    if lap_in_stint <= warmup_laps:
        return penalty * (1 - (lap_in_stint - 1) / warmup_laps)
    return 0

In [ ]:
def lap_time(model, track, lap, lap_in_stint, total_laps):
    base = BASE_LAP_TIMES[track]

    degradation_factor = model.predict(np.array([lap_in_stint]))[0]
    degradation_penalty = (1 - degradation_factor) * 5

    fuel_pen = fuel_time_penalty(lap, total_laps)
    warmup_pen = tyre_warmup_penalty(lap_in_stint)

    return base + degradation_penalty + fuel_pen + warmup_pen

In [ ]:
def simulate_race_time(strategy_fn, model, track, total_laps, *pit_params, pit_loss=20):
    laps = []
    stint_start = 1
    pit_stops = list(pit_params)

    current_stint_lap = 1
    pit_index = 0

    for lap in range(1, total_laps + 1):
        if pit_index < len(pit_stops) and lap == pit_stops[pit_index]:
            laps.append(pit_loss)
            current_stint_lap = 1
            pit_index += 1
            continue

        laps.append(lap_time(model, track, lap, current_stint_lap, total_laps))
        current_stint_lap += 1

    return np.sum(laps)

In [ ]:
def monte_carlo_race_time(
    strategy_fn,
    compound,
    track,
    driver_style,
    total_laps,
    pit_params,
    alpha_std=0.005,
    n_sim=1000,
    pit_loss=20
):
    race_times = []

    base_model = create_context_aware_model(compound, track, driver_style)
    alpha_mean = base_model.alpha

    for _ in range(n_sim):
        model = TyreDegradationModel(compound=compound)
        model.alpha = np.random.normal(alpha_mean, alpha_std)

        race_time = simulate_race_time(
            strategy_fn,
            model,
            track,
            total_laps,
            *pit_params,
            pit_loss=pit_loss
        )

        race_times.append(race_time)

    return np.array(race_times)

In [ ]:
def compare_strategies_race_time(track, driver, compound="Soft",
                                 total_laps=70,
                                 one_stop_pit=35,
                                 two_stop_pits=(23, 46),
                                 alpha_std=0.005,
                                 n_sim=2000):

    style = DRIVER_STYLE_MAP[driver]

    one_times = monte_carlo_race_time(
        one_stop_strategy,
        compound,
        track,
        style,
        total_laps,
        (one_stop_pit,),
        alpha_std,
        n_sim
    )

    two_times = monte_carlo_race_time(
        two_stop_strategy,
        compound,
        track,
        style,
        total_laps,
        two_stop_pits,
        alpha_std,
        n_sim
    )

    win_prob = np.mean(two_times < one_times)

    summary = pd.DataFrame([
        ["One-stop", one_times.mean(), one_times.std()],
        ["Two-stop", two_times.mean(), two_times.std()]
    ], columns=["Strategy", "Mean Race Time", "Std Dev"])

    best = "Two-stop" if two_times.mean() < one_times.mean() else "One-stop"

    return {
        "track": track,
        "driver": driver,
        "best_strategy": best,
        "two_stop_win_probability": win_prob,
        "comparison_table": summary
    }

In [ ]:
race_example = compare_strategies_race_time(
    track="Hungaroring",
    driver="Lewis Hamilton"
)

race_example

In [ ]:
SAFETY_CAR_PROBABILITY = {
    "Monaco": 0.6,
    "Singapore": 0.7,
    "Baku": 0.5,
    "Bahrain": 0.3,
    "Barcelona": 0.25,
    "Silverstone": 0.2,
    "Monza": 0.15,
    "Spa": 0.25,
    "Mexico": 0.3,
    "AbuDhabi": 0.2,
    "Suzuka": 0.25,
    "Hungaroring": 0.35
}

In [ ]:
def sample_safety_car(track, total_laps):
    prob = SAFETY_CAR_PROBABILITY.get(track, 0.3)

    if np.random.rand() < prob:
        return np.random.randint(5, total_laps - 5)
    return None

In [ ]:
def simulate_race_time_with_sc(
    strategy_fn,
    model,
    track,
    total_laps,
    *pit_params,
    base_pit_loss=20,
    sc_pit_loss=10
):
    sc_lap = sample_safety_car(track, total_laps)

    laps = []
    current_stint_lap = 1
    pit_index = 0
    pit_stops = list(pit_params)

    for lap in range(1, total_laps + 1):

        if pit_index < len(pit_stops) and lap == pit_stops[pit_index]:

            if sc_lap is not None and abs(lap - sc_lap) <= 1:
                laps.append(sc_pit_loss)
            else:
                laps.append(base_pit_loss)

            current_stint_lap = 1
            pit_index += 1
            continue

        lap_t = lap_time(model, track, lap, current_stint_lap, total_laps)

        if sc_lap is not None and lap == sc_lap:
            lap_t *= 1.4

        laps.append(lap_t)
        current_stint_lap += 1

    return np.sum(laps)

In [ ]:
def monte_carlo_race_time_sc(
    strategy_fn,
    compound,
    track,
    driver_style,
    total_laps,
    pit_params,
    alpha_std=0.005,
    n_sim=1000
):
    race_times = []

    base_model = create_context_aware_model(compound, track, driver_style)
    alpha_mean = base_model.alpha

    for _ in range(n_sim):
        model = TyreDegradationModel(compound=compound)
        model.alpha = np.random.normal(alpha_mean, alpha_std)

        race_time = simulate_race_time_with_sc(
            strategy_fn,
            model,
            track,
            total_laps,
            *pit_params
        )

        race_times.append(race_time)

    return np.array(race_times)

In [ ]:
def compare_strategies_with_sc(track, driver, compound="Soft",
                               total_laps=70,
                               one_stop_pit=35,
                               two_stop_pits=(23, 46),
                               alpha_std=0.005,
                               n_sim=2000):

    style = DRIVER_STYLE_MAP[driver]

    one_times = monte_carlo_race_time_sc(
        one_stop_strategy,
        compound,
        track,
        style,
        total_laps,
        (one_stop_pit,),
        alpha_std,
        n_sim
    )

    two_times = monte_carlo_race_time_sc(
        two_stop_strategy,
        compound,
        track,
        style,
        total_laps,
        two_stop_pits,
        alpha_std,
        n_sim
    )

    win_prob = np.mean(two_times < one_times)

    summary = pd.DataFrame([
        ["One-stop", one_times.mean(), one_times.std()],
        ["Two-stop", two_times.mean(), two_times.std()]
    ], columns=["Strategy", "Mean Race Time", "Std Dev"])

    best = "Two-stop" if two_times.mean() < one_times.mean() else "One-stop"

    return {
        "track": track,
        "driver": driver,
        "best_strategy": best,
        "two_stop_win_probability": win_prob,
        "comparison_table": summary
    }

In [ ]:
sc_example = compare_strategies_with_sc(
    track="Hungaroring",
    driver="Lewis Hamilton"
)

sc_example

In [ ]:
OVERTAKE_DIFFICULTY = {
    "Monaco": 0.95,
    "Singapore": 0.85,
    "Baku": 0.6,
    "Bahrain": 0.55,
    "Barcelona": 0.7,
    "Silverstone": 0.5,
    "Monza": 0.4,
    "Spa": 0.45,
    "Mexico": 0.5,
    "AbuDhabi": 0.6,
    "Suzuka": 0.7,
    "Hungaroring": 0.9
}

In [ ]:
def traffic_time_penalty(track, base_loss=1.2):
    difficulty = OVERTAKE_DIFFICULTY.get(track, 0.6)

    if np.random.rand() < difficulty:
        return np.random.exponential(base_loss)
    return 0

In [ ]:
def pit_rejoin_penalty(track, mean_loss=3.0):
    difficulty = OVERTAKE_DIFFICULTY.get(track, 0.6)
    return np.random.exponential(mean_loss * difficulty)

In [ ]:
def simulate_race_time_full(
    strategy_fn,
    model,
    track,
    total_laps,
    *pit_params,
    base_pit_loss=20,
    sc_pit_loss=10
):
    sc_lap = sample_safety_car(track, total_laps)

    laps = []
    current_stint_lap = 1
    pit_index = 0
    pit_stops = list(pit_params)

    for lap in range(1, total_laps + 1):

        # PIT STOP
        if pit_index < len(pit_stops) and lap == pit_stops[pit_index]:

            if sc_lap is not None and abs(lap - sc_lap) <= 1:
                pit_time = sc_pit_loss
            else:
                pit_time = base_pit_loss

            pit_time += pit_rejoin_penalty(track)

            laps.append(pit_time)
            current_stint_lap = 1
            pit_index += 1
            continue

        # NORMAL LAP
        lap_t = lap_time(model, track, lap, current_stint_lap, total_laps)

        # SAFETY CAR SLOWDOWN
        if sc_lap is not None and lap == sc_lap:
            lap_t *= 1.4

        # TRAFFIC EFFECT
        lap_t += traffic_time_penalty(track)

        laps.append(lap_t)
        current_stint_lap += 1

    return np.sum(laps)

In [ ]:
def monte_carlo_race_time_full(
    strategy_fn,
    compound,
    track,
    driver_style,
    total_laps,
    pit_params,
    alpha_std=0.005,
    n_sim=1000
):
    race_times = []

    base_model = create_context_aware_model(compound, track, driver_style)
    alpha_mean = base_model.alpha

    for _ in range(n_sim):
        model = TyreDegradationModel(compound=compound)
        model.alpha = np.random.normal(alpha_mean, alpha_std)

        race_time = simulate_race_time_full(
            strategy_fn,
            model,
            track,
            total_laps,
            *pit_params
        )

        race_times.append(race_time)

    return np.array(race_times)

In [ ]:
def compare_strategies_full(track, driver, compound="Soft",
                            total_laps=70,
                            one_stop_pit=35,
                            two_stop_pits=(23, 46),
                            alpha_std=0.005,
                            n_sim=3000):

    style = DRIVER_STYLE_MAP[driver]

    one_times = monte_carlo_race_time_full(
        one_stop_strategy, compound, track, style,
        total_laps, (one_stop_pit,), alpha_std, n_sim
    )

    two_times = monte_carlo_race_time_full(
        two_stop_strategy, compound, track, style,
        total_laps, two_stop_pits, alpha_std, n_sim
    )

    win_prob = np.mean(two_times < one_times)

    summary = pd.DataFrame([
        ["One-stop", one_times.mean(), one_times.std()],
        ["Two-stop", two_times.mean(), two_times.std()]
    ], columns=["Strategy", "Mean Race Time", "Std Dev"])

    best = "Two-stop" if two_times.mean() < one_times.mean() else "One-stop"

    return {
        "track": track,
        "driver": driver,
        "best_strategy": best,
        "two_stop_win_probability": win_prob,
        "comparison_table": summary
    }

In [ ]:
full_example = compare_strategies_full(
    track="Monaco",
    driver="Lewis Hamilton"
)

full_example